In [2]:
import os
import numpy as np
from collections import defaultdict
import math

from conlleval2 import evaluate

In [30]:
def get_emission_dict(file_path):
    
    e = {}
    y_count = {}
    emission_dict = {}
    
    train_data = open(file_path, 'r', encoding="utf-8")
    lines = train_data.readlines()
    
    all_x = set()
    all_y = set()

    for line in lines:
        # "x pos y\n"
        line = line.strip()

        if len(line) > 0:
            x, pos, y = line.split(" ")
            all_x.add(x)
            all_y.add(y)
            y_count[y] = y_count.get(y,0) + 1
            e[(x, y)]  = e.get((x,y),0) + 1
            e[(pos, y)] = e.get((pos,y),0) + 1

    for x, y in e.keys():
        # x here can be x or pos tag
        key = "emission:" + y + "+" + x
        emission_dict[key] = math.log(e[(x, y)] / y_count[y])
    
    return emission_dict, list(all_y)

train_path = "full/train"
emission_dict, states = get_emission_dict(train_path)

In [31]:
emission_dict

{'emission:O+The': -4.333515843240958,
 'emission:O+DT': -2.2045841023613524,
 'emission:O+official': -7.166729187297174,
 'emission:O+JJ': -2.617071711239342,
 'emission:O+cause': -8.776167099731275,
 'emission:O+NN': -1.8272698774179623,
 'emission:O+of': -3.518671727703493,
 'emission:O+IN': -1.9894501491261936,
 'emission:O+death': -7.166729187297174,
 'emission:O+has': -4.649032714686183,
 'emission:O+VBZ': -3.5639524322366496,
 'emission:O+not': -5.97280671882474,
 'emission:O+RB': -3.8525431826246486,
 'emission:O+been': -5.755742213586912,
 'emission:O+VBN': -3.3294297280649645,
 'emission:O+officially': -8.370701991623111,
 'emission:O+determined': -8.370701991623111,
 'emission:O+,': -2.6427690567346263,
 'emission:O+but': -6.211217742269738,
 'emission:O+CC': -3.7199212943829667,
 'emission:O+investigators': -9.46931428029122,
 'emission:O+NNS': -2.5233002891919925,
 'emission:O+believe': -9.46931428029122,
 'emission:O+VBP': -4.122206749573751,
 'emission:O+the': -2.8532490

In [32]:
def get_feature_count(x, y, feature_dict):
    n = len(x)
    feature_count = {}
    
    for i in range(n):
        formatted_word = x[i]
        emission_key1 = "emission:" + y[i] + "+" + x[i][0]
        emission_key2 = "emission:" + y[i] + "+" + x[i][1]
        feature_count[emission_key1] = feature_count.get(emission_key1, 0) + 1
        feature_count[emission_key2] = feature_count.get(emission_key2, 0) + 1
    
    
    updated_y = ["start"] + y + ["stop"]
    for i in range(1, n+2):
        prev_y = updated_y[i-1]
        y_i = updated_y[i]
        transition_key = "transition:"+prev_y+"+"+y_i
        feature_count[transition_key] = feature_count.get(transition_key, 0) + 1
    
    return feature_count

In [33]:
def get_resulting_dict(file_path, emission_dict):

    t = {}
    y_count = {}
    
    train_data = open(file_path, 'r', encoding="utf-8")
    lines = train_data.readlines()
    start = "start"
    all_y = set(["start", "stop"])

    for line in lines:
        # "x pos y\n"
        line = line.strip()
        if len(line) <= 0:
            t[(start, "stop")] = t.get((start,"stop"),0) + 1
            start = "start"
            y_count[start] = y_count.get(start,0) + 1
        else:
            x, pos, y = line.split(" ")
            t[(start, y)] = t.get((start,y),0) + 1
            y_count[y] = y_count.get(y,0) + 1
            start = y
            all_y.add(y)

    for start, end in t.keys():
        key = "transition:" + start + "+" + end
        emission_dict[key] = math.log(t[(start, end)] / y_count[start])

    return emission_dict

resulting_dict = get_resulting_dict(train_path, emission_dict)

In [34]:
resulting_dict

{'emission:O+The': -4.333515843240958,
 'emission:O+DT': -2.2045841023613524,
 'emission:O+official': -7.166729187297174,
 'emission:O+JJ': -2.617071711239342,
 'emission:O+cause': -8.776167099731275,
 'emission:O+NN': -1.8272698774179623,
 'emission:O+of': -3.518671727703493,
 'emission:O+IN': -1.9894501491261936,
 'emission:O+death': -7.166729187297174,
 'emission:O+has': -4.649032714686183,
 'emission:O+VBZ': -3.5639524322366496,
 'emission:O+not': -5.97280671882474,
 'emission:O+RB': -3.8525431826246486,
 'emission:O+been': -5.755742213586912,
 'emission:O+VBN': -3.3294297280649645,
 'emission:O+officially': -8.370701991623111,
 'emission:O+determined': -8.370701991623111,
 'emission:O+,': -2.6427690567346263,
 'emission:O+but': -6.211217742269738,
 'emission:O+CC': -3.7199212943829667,
 'emission:O+investigators': -9.46931428029122,
 'emission:O+NNS': -2.5233002891919925,
 'emission:O+believe': -9.46931428029122,
 'emission:O+VBP': -4.122206749573751,
 'emission:O+the': -2.8532490

In [35]:
def viterbi_algo(x, states, f):
    scores = np.full((len(x), len(states)), -np.inf)
    parents = np.full((len(x), len(states)), 0, dtype=int)
    
    for i in range(len(states)):
        emission_key1 = "emission:" + states[i] + "+" + x[0].split()[0]
        emission_key2 = "emission:" + states[i] + "+" + x[0].split()[1]
        transition_key = "transition:" + "start" + "+" + states[i]
        scores[0, i] = f.get(emission_key1, -10e8) + f.get(emission_key2, -10e8) + f.get(transition_key, -10e8)
    
    for i in range(1, len(x)):
        for j in range(len(states)):
            for k in range(len(states)):
                emission_key1 = "emission:" + states[k] + "+" + x[i].split()[0]
                emission_key2 = "emission:" + states[k] + "+" + x[i].split()[1]
                transition_key = "transition:" + states[j] + "+" + states[k]
                overall_score = scores[i-1, j] + f.get(emission_key1, -10e8) + f.get(emission_key2, -10e8) + f.get(transition_key, -10e8)

                if overall_score > scores[i, k]:
                    scores[i, k] = overall_score
                    parents[i,k] = j
    
    best_score = -np.inf
    best_parent = None
    
    for i in range(len(states)):
        t_feature = "transition:" + states[i] + "+" + "stop"
        total = scores[len(x)-1, i] + f.get(t_feature, -10**8)
        
        if total > best_score:
            best_score = total
            best_parent = i
    
    best_state = [states[best_parent]]
    prev_parent = best_parent
    for i in range(len(x)-1, 0, -1):
        prev_parent = parents[i, prev_parent]
        output = states[prev_parent]
        best_state = [output] + best_state
    return best_state

In [36]:
def get_prediction(file_dir,resulting_dict):
    train_path = os.path.join(file_dir, "train")
    input_path = os.path.join(file_dir, "dev.in")
    
    test_set = open(input_path, 'r', encoding="utf-8")
    lines = test_set.readlines()
    
    sequences = []
    sequence = []
    for line in lines:
        if line == '\n':
            sequences.append(sequence)
            sequence = []
            continue

        line = line.replace('\n', '')
        sequence.append(line)

    out_path = os.path.join(file_dir, "dev.p5.CRF.f3.out")
    out_file = open(out_path, "w", encoding="utf-8")
    
    for x in sequences:
        predicted = viterbi_algo(x, states, resulting_dict)
        for i in range(len(x)):
            out_file.write(x[i] + ' ' + predicted[i] + '\n')
        out_file.write('\n')
    out_file.close()
    
    print("Complete prediction for dataset")

file_dir = "full"
get_prediction(file_dir,resulting_dict)

Complete prediction for dataset


In [37]:
def compute_crf_score(x, y, feature_dict):
    feature_count = {}
    for i in range(len(x)):
        formatted_word = x[i]
#         print(formatted_word)
        emission_key = "emission:"+str(y[i])+"+"+str(formatted_word[0])
        emission_key2 = "emission:"+str(y[i])+"+"+str(formatted_word[1])
        feature_count[emission_key] = feature_count.get(emission_key, 0) + 1
        feature_count[emission_key2] = feature_count.get(emission_key2, 0) + 1
    updated_y = ["start"]
    updated_y.extend(y)
    updated_y.append("stop")
    for i in range(0, len(x)+1):
        prev_y = updated_y[i]
        y_i = updated_y[i+1]
        transition_key = "transition:"+str(prev_y)+"+"+str(y_i)
        feature_count[transition_key] = feature_count.get(transition_key, 0) + 1
    score = 0
    for k, v in feature_count.items():
        weight = feature_dict[k]
        score += weight * v
    return score

def compute_forward_score(x, feature_dict, states):
    forward_scores = np.zeros((len(x), len(states)))
    threshold = 700

    for i in range(len(states)):
        transition_key = "transition:START+"+states[i]
        transition_score = feature_dict.get(transition_key, -9999999)
        emission_key = "emission:"+states[i]+"+"+x[0][0]
        emission_score = feature_dict.get(emission_key, -9999999)
        emission_key2 = "emission:"+states[i]+"+"+x[0][1]
        emission_score2 = feature_dict.get(emission_key, -9999999)
        forward_scores[0, i] = transition_score + emission_score + emission_score2
    
    for i in range(1, len(x)):
        for k in range(len(states)):
            temp_score = 0
            for j in range(len(states)):
                transition_key = "transition:"+states[j]+"+"+states[k]
                transition_score = feature_dict.get(transition_key, -9999999)
                emission_key = "emission:"+states[k]+"+"+x[i][0]
                emission_score = feature_dict.get(emission_key, -9999999)
                emission_key2 = "emission:"+states[k]+"+"+x[i][1]
                emission_score2 = feature_dict.get(emission_key, -9999999)
                score = emission_score + transition_score + forward_scores[i-1,j] + emission_score2
                if score > threshold:
                    score = threshold
                temp_score += np.exp(score)
                
            forward_scores[i, k] = np.log(temp_score) if temp_score else -9999999

    forward_prob = 0
    for j in range(len(states)):
        transition_key = "transition:"+states[j]+"+stop"
        transition_score = feature_dict.get(transition_key, -9999999)
        score = transition_score + forward_scores[len(x)-1,j]
        if score > threshold:
            score = threshold
        overall_score = np.exp(score)
        forward_prob += overall_score
    if forward_prob > 0:
        alpha = np.log(forward_prob)
    else:
        alpha = threshold
        
    return forward_scores, alpha

def compute_backward_score(x, feature_dict, states):
    
    backward_scores = np.zeros((len(x), len(states)))
    threshold = 700
    
    for i in range(len(states)):
        transition_key = "transition:"+ states[i] +"+stop"
        transition_score = feature_dict.get(transition_key, -9999999)
        backward_scores[len(x)-1, i] = transition_score

    for i in range(len(x)-1,0,-1):
        for j  in range(len(states)):
            temp_score = 0 
            for k in range(len(states)):
                transition_key = "transition:" + states[j] + "+" + states[k]
                emission_key = "emission:"+ states[k] + "+" + x[i][0]
                emission_key2 = "emission:"+ states[k] + "+" + x[i][1] 
                transition_score = feature_dict.get(transition_key, -9999999)
                emission_score = feature_dict.get(emission_key, -9999999)
                emission_score2 = feature_dict.get(emission_key2,-9999999)
                temp_score += np.exp(min(emission_score + transition_score + emission_score2 + \
                                                             backward_scores[i, k], threshold))
            
            if temp_score!=0:
                backward_scores[i-1, j] = math.log(temp_score)
            else:
                backward_scores[i-1, j] = -9999999
    
    backward_prob = 0
    for i in range(len(states)):
        transition_key = "transition:" + "start" + "+" + states[i]
        emission_key = "emission:" + states[i] + "+" + x[0][0]
        emission_key2 = "emission:" + states[i] + "+" + x[0][1]
        transition_score = feature_dict.get(transition_key, -9999999)
        emission_score = feature_dict.get(emission_key, -9999999)
        emission_score2 = feature_dict.get(emission_key2, -9999999)
        overall_score = math.exp(min(emission_score + transition_score+ emission_score2 + backward_scores[0, i], threshold))
        backward_prob += overall_score
        
    if backward_prob!=0:
        beta = math.log(backward_prob)
    else:
        beta = -threshold    

    return backward_scores, beta

def forward_backward(x, feature_dict, states):
    
    threshold = 700
    
    forward_scores, alpha = compute_forward_score(x, feature_dict, states)

    forward_prob = math.exp(min(alpha, threshold))
    backward_scores, beta = compute_backward_score(x, feature_dict, states)
    backward_prob = math.exp(min(beta, threshold))
    feature_expected_counts = {}

    for i in range(len(x)):
        for j in range(len(states)):
            emission_key = "emission:" + states[j] + "+" + x[i][1]
            feature_expected_counts[emission_key] = feature_expected_counts.get(emission_key, 0.0) + \
                        math.exp(min(forward_scores[i, j] + backward_scores[i, j] - alpha, threshold))
            emission_key2 = "emission:" + states[j] + "+" + x[i][0]
            feature_expected_counts[emission_key2] = feature_expected_counts.get(emission_key2, 0.0) + \
                        math.exp(min(forward_scores[i, j] + backward_scores[i, j] - alpha, threshold))
        
    for i in range(len(states)):
        start_transition_key = "transition:" + "start" + "+" + states[i]
        feature_expected_counts[start_transition_key] = feature_expected_counts.get(start_transition_key, 0.0) \
                                + math.exp(min(forward_scores[0, i] + backward_scores[0, i] - alpha, threshold))
        stop_transition_key = "transition:" + states[i] + "+"  + "stop"
        feature_expected_counts[stop_transition_key] = feature_expected_counts.get(stop_transition_key, 0.0) + \
                    math.exp(min(forward_scores[len(x)-1, i] + backward_scores[len(x)-1, i] - alpha, threshold))
    
    for i in range(len(states)):
        for j in range(len(states)):
            transition_key = "transition:" + states[i] + "+"  + states[j]
            transition_score = feature_dict.get(transition_key, -9999999) 
            total = 0
            for k in range(len(x)-1):
                emission_key =  "emission:" + states[j] + "+"  + x[k+1][0]
                emission_score = feature_dict.get(emission_key, -9999999)
                emission_key2 =  "emission:" + states[j] + "+"  + x[k+1][1]
                emission_score2 = feature_dict.get(emission_key2, -9999999)

                total += np.exp(min(forward_scores[k, i] + backward_scores[k+1, j] + transition_score + \
                                                        emission_score + emission_score2 - alpha,threshold))

            feature_expected_counts[transition_key] = total
    
    return feature_expected_counts

In [38]:
def get_dataset(path):
    input_sequences = []
    input_labels = []
    with open(path) as f:
        lines = f.readlines()
        sentence = []
        labels = []
        for line in lines:
            formatted_line = line.strip()
            if len(formatted_line) > 0:
                split_data = formatted_line.split(" ")
                x, y = split_data[0:2], split_data[2]
                sentence.append(x)
                labels.append(y)
            else:
                input_sequences.append(sentence)
                input_labels.append(labels)
                sentence = []
                labels = []
    return input_sequences, input_labels
train_inputs, train_labels = get_dataset("full/train")

In [39]:
for x in resulting_dict:
    print(x)

emission:O+The
emission:O+DT
emission:O+official
emission:O+JJ
emission:O+cause
emission:O+NN
emission:O+of
emission:O+IN
emission:O+death
emission:O+has
emission:O+VBZ
emission:O+not
emission:O+RB
emission:O+been
emission:O+VBN
emission:O+officially
emission:O+determined
emission:O+,
emission:O+but
emission:O+CC
emission:O+investigators
emission:O+NNS
emission:O+believe
emission:O+VBP
emission:O+the
emission:O+36-year-old
emission:O+writer
emission:O+died
emission:O+VBD
emission:O+from
emission:O+a
emission:O+self-inflicted
emission:O+gunshot
emission:O+wound
emission:O+.
emission:B-per+Mr.
emission:B-per+NNP
emission:I-per+Omi
emission:I-per+NNP
emission:O+called
emission:O+for
emission:O+stronger
emission:O+JJR
emission:O+political
emission:O+will
emission:O+by
emission:O+Asian
emission:O+governments
emission:O+to
emission:O+TO
emission:O+stop
emission:O+VB
emission:O+spread
emission:O+disease
emission:O+This
emission:O+is
emission:O+second
emission:O+U.N.-Congolese
emission:O+offen

emission:I-tim+NNS
emission:O+outstanding
emission:O+renewed
emission:O+appeal
emission:O+ceasefire
emission:O+guerrillas
emission:O+intensified
emission:O+change
emission:B-org+Supreme
emission:I-org+Court
emission:O+nominations
emission:O+She
emission:O+framed
emission:B-geo+Sudan
emission:B-geo+western
emission:I-geo+Darfur
emission:O+faltering
emission:O+learned
emission:O+unique
emission:O+subspecies
emission:B-org+Plains
emission:I-org+Zebra
emission:B-org+Quagga
emission:I-org+Project
emission:O+started
emission:I-org+African
emission:B-per+Reinhold
emission:I-per+Rau
emission:O+animal
emission:O+ethnocultural
emission:O+divide
emission:O+persisted
emission:O+turbulent
emission:O+politics
emission:O+youth
emission:B-org+Labor
emission:O+showed
emission:O+4.6
emission:O+With
emission:O+races
emission:B-tim+season
emission:B-per+Alonso
emission:O+third-place
emission:O+finish
emission:O+champion
emission:B-geo+Ferrari
emission:B-per+Michael
emission:I-per+Schumacher
emission:B-geo

emission:I-geo+York
emission:O+79-year-old
emission:O+unspecified
emission:B-per+Pope
emission:I-per+Benedict
emission:O+marked
emission:O+By
emission:O+defeated
emission:O+remnants
emission:B-org+LTTE
emission:O+concession
emission:I-org+Caracas
emission:I-org+Television
emission:B-geo+Marine
emission:B-org+Naval
emission:I-org+Criminal
emission:I-org+Investigative
emission:O+hostages
emission:O+three-year-old
emission:O+boy
emission:B-geo+FARC
emission:B-per+Rahimgul
emission:I-per+Sarawan
emission:B-org+KFC
emission:O+inspired
emission:I-per+Sanders
emission:O+association
emission:O+Chief
emission:I-per+del
emission:I-per+Ponte
emission:B-per+Slobodan
emission:O+medicines
emission:O+Susilo
emission:B-per+Bambang
emission:I-per+Yudhoyono
emission:O+anti-corruption
emission:O+bowlers
emission:B-geo+Makhaya
emission:I-geo+Ntini
emission:B-per+Dale
emission:I-per+Steyn
emission:B-org+Steyn
emission:O+wickets
emission:O+allowing
emission:O+runs
emission:B-geo+Ntini
emission:O+finished
em

In [40]:
from scipy.optimize import fmin_l_bfgs_b

def compute_gradients_with_reg(train_inputs, train_labels, f, states, nabla = 0):
    feature_gradients = {}
    for i in range(len(train_labels)):
        x = train_inputs[i]
        y = train_labels[i]
        feature_expected_counts = forward_backward(x, f, states)
        actual_counts = get_feature_count(x, y, f)
        
        for k, v in feature_expected_counts.items():
            feature_gradients[k] = feature_gradients.get(k, 0) + v
            
        for k, v in actual_counts.items():
            feature_gradients[k] = feature_gradients.get(k, 0) - v

    for k, v in f.items():
        feature_gradients[k] = feature_gradients.get(k,0) + 2*nabla*f[k]

    return feature_gradients

def compute_crf_loss_with_reg(input_sequences, input_labels, feature_dict, states, nabla=0):
    loss = 0
    for i in range(len(input_sequences)):
        first_term = compute_crf_score(input_sequences[i], input_labels[i], feature_dict)
        _, forward_score = compute_forward_score(input_sequences[i], feature_dict, states)
        loss += (first_term - forward_score) * -1

    reg_loss = 0
    for feature_key in feature_dict:
        reg_loss += feature_dict[feature_key]**2
    reg_loss = nabla * reg_loss
    loss += reg_loss
    return loss

def callbackF(w):
    '''
    This function will be called by "fmin_l_bfgs_b"
    Arg:
        w: weights, numpy array
    '''
    loss = compute_crf_loss_with_reg(train_inputs,train_labels,resulting_dict,states,0.1) 
    print('Loss:{0:.4f}'.format(loss))
def get_loss_grad(w, *args): 
    '''
    This function will be called by "fmin_l_bfgs_b"
    Arg:
        w: weights, numpy array
    Returns:
        loss: loss, float
        grads: gradients, numpy array
    '''
    train_inputs,train_labels,f,states = args
    for i,k in enumerate(f.keys()):
        f[k] = w[i]
    
    loss = compute_crf_loss_with_reg(train_inputs, train_labels, f, states, 0.1)
    grads = compute_gradients_with_reg(train_inputs, train_labels, f, states, 0.1)
    np_grads = np.zeros(len(f))
    for i,k in enumerate(f.keys()):
        np_grads[i] = grads[k]
    grads = np_grads
    return loss, grads

init_w = np.zeros(len(resulting_dict))
result = fmin_l_bfgs_b(get_loss_grad, init_w, args=(train_inputs, train_labels, resulting_dict, states), pgtol=0.01, callback=callbackF)

Loss:-208811419.3752
Loss:-345589525.0000


In [42]:
def numpy_to_dict(w, f, reverse = False):
    for i,k in enumerate(f.keys()):
        f[k] = w[i]
    return f

In [44]:
resulting_dict = numpy_to_dict(result[0], resulting_dict)
get_prediction(file_dir,resulting_dict)

Complete prediction for dataset


In [3]:
def eval(pred,gold):
    f_pred = open(pred,encoding = 'utf-8')
    f_gold = open(gold,encoding = 'utf-8')
    data_pred = f_pred.readlines()
    data_gold = f_gold.readlines()
    gold_tags = list()
    pred_tags = list()
    for sentence in range(len(data_pred)):
        words_pred = data_pred[sentence].strip().split(' ')
        words_gold = data_gold[sentence].strip().split(' ')  
        if len(words_gold)==1:
            continue
        gold_tags.append(words_gold[2])
        pred_tags.append(words_pred[2])
    return gold_tags,pred_tags

true_path = "full/dev.out"
pred_path = "full/dev.p5.CRF.f3.out"

g_tags, p_tags = eval(true_path, pred_path)
print(evaluate(g_tags,p_tags,verbose=True))

processed 2097 tokens with 172 phrases; found: 236 phrases; correct: 121.
accuracy:  78.57%; (non-O)
accuracy:  91.32%; precision:  51.27%; recall:  70.35%; FB1:  59.31
              art: precision:   0.00%; recall:   0.00%; FB1:   0.00  3
              eve: precision:   0.00%; recall:   0.00%; FB1:   0.00  1
              geo: precision:  65.88%; recall:  77.78%; FB1:  71.34  85
              gpe: precision:  40.00%; recall:  76.92%; FB1:  52.63  25
              nat: precision:   0.00%; recall:   0.00%; FB1:   0.00  2
              org: precision:  37.14%; recall:  61.90%; FB1:  46.43  35
              per: precision:  34.38%; recall:  57.89%; FB1:  43.14  32
              tim: precision:  58.49%; recall:  67.39%; FB1:  62.63  53
((51.271186440677965, 70.34883720930233, 59.31372549019608), 0)
